In [ ]:
!wget https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz

--2020-11-08 05:07:00--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.68.30
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.68.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz.1’

GoogleNews-vectors- 100%[===================>]   1.53G  43.1MB/s    in 39s     

2020-11-08 05:07:39 (40.7 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz.1’ saved [1647046227/1647046227]



In [ ]:


import gensim
from gensim import utils
import numpy as np
from nltk import word_tokenize
from nltk import download

from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler


In [ ]:
word_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)


In [ ]:
download('punkt')
def preprocess(text5):
    text5 = text5.lower()
    doc = word_tokenize(text5)
    return doc



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def document_vector(word2vec_model, doc):
    doc = [word for word in doc if word in word2vec_model.vocab]
    if(len(doc)==0):
      return [0]*300
    return np.mean(word2vec_model[doc], axis=0)



In [ ]:
file1 = open('Subj.x.train.txt', 'r',encoding='ISO-8859-1') 
texts = file1.readlines() 

"these four queens became germany's most popular drag performers and have been busy fertilizing the german cultural scene . \n"

In [ ]:
file3 = open('Subj.x.test.txt', 'r',encoding='ISO-8859-1') 
texts2 = file3.readlines()

'jack black will play dewey finn , a depressed musician that was recently fired from his band . \n'

In [ ]:
from sklearn.preprocessing import LabelEncoder

file1 = open('Subj.y.train.txt', 'r') 
Lines = file1.readlines() 
count = 0
target1=[]
for line in Lines: 
    target1.append(line)
le = LabelEncoder()
le.fit(["SUBJ\n","OBJ\n"])
list(le.classes_)
labels=le.transform(target1)
labels[:29]

array([0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0])

In [ ]:
texts, y = texts, labels
corpus = [preprocess(text) for text in texts]

In [ ]:
test_corpus = [preprocess(text3) for text3 in texts2]

In [ ]:

x =[]
for doc in corpus: 
      x.append(document_vector(word_model, doc))
X = np.array(x)
X.shape, len(y)


((9000, 300), 9000)

In [ ]:
X[90]

In [ ]:
test_x =[]
for doc1 in test_corpus: 
    test_x.append(document_vector(word_model, doc1))
test_X = np.array(test_x) 
np.savetxt('subjtest_documents_vectors2.txt', test_X)
test_X.shape


(1000, 300)

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [ ]:
train_x=X
train_y=y
X_train, X_val, Y_train, Y_val = train_test_split(train_x, train_y, test_size=0.20, random_state=42)
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
Y_train=tf.convert_to_tensor(Y_train, dtype=tf.float32)
X_val = tf.convert_to_tensor(X_val, dtype=tf.float32)
Y_val=tf.convert_to_tensor(Y_val, dtype=tf.float32)

In [ ]:
from sklearn.linear_model import LogisticRegressionCV

clf = LogisticRegressionCV(cv=6, max_iter=1000,random_state=0,multi_class='ovr').fit(X_train, Y_train)
score = clf.score(X_val, Y_val)

print("Accuracy:", score)

Accuracy: 0.9061111111111111


In [ ]:
from sklearn.preprocessing import StandardScaler
scale=StandardScaler()
scale.fit(X_train)
X_train=scale.transform(X_train)
X_val=scale.transform(X_val)

In [ ]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(max_iter=1000,random_state=0,multi_class='ovr')
classifier.fit(X_train, Y_train)
score = classifier.score(X_val, Y_val)

print("Accuracy:", score)

Accuracy: 0.905


In [ ]:
X_test=tf.convert_to_tensor(test_X, dtype=tf.float32)

In [ ]:
ypred=clf.predict(X_test)

In [ ]:
COUNT=0
for x in ypred:
  if(x==0):
    COUNT+=1

print(COUNT)

483


In [ ]:
print(ypred[:300])

[0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0.
 1. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1. 0. 0.
 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1.
 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0.
 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0.
 0. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1.
 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1.
 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0.
 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 1. 0.
 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0.
 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0.
 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0.
 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1.]


In [ ]:
print(ypred[:50])

In [ ]:
model = Sequential()
model.add(Dense(300, input_dim=300, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
history=model.fit(X_train, Y_train, validation_data=(X_val,Y_val), epochs=5, batch_size=10)

Epoch 1/5
720/720 [==============================] - 1s 2ms/step - loss: 0.2896 - accuracy: 0.8864 - val_loss: 0.2503 - val_accuracy: 0.9011
Epoch 2/5
720/720 [==============================] - 1s 2ms/step - loss: 0.2243 - accuracy: 0.9117 - val_loss: 0.2567 - val_accuracy: 0.8956
Epoch 3/5
720/720 [==============================] - 1s 2ms/step - loss: 0.2046 - accuracy: 0.9176 - val_loss: 0.2475 - val_accuracy: 0.9000
Epoch 4/5
720/720 [==============================] - 1s 2ms/step - loss: 0.1928 - accuracy: 0.9237 - val_loss: 0.2416 - val_accuracy: 0.9028
Epoch 5/5
720/720 [==============================] - 1s 2ms/step - loss: 0.1850 - accuracy: 0.9258 - val_loss: 0.2363 - val_accuracy: 0.9056


In [ ]:
o=model.evaluate(X_val,Y_val)

63/63 [==============================] - 0s 1ms/step - loss: 0.5412 - accuracy: 0.8135


In [ ]:
y_pred=model.predict_classes(X_test)

In [ ]:
y_pred2=model.predict_classes(X_val)

In [ ]:
print(y_pred)

In [ ]:
count=0
for x in ypred:
  if(x==0):
    count+=1
print(count)

483


In [ ]:
Y_test=[]
for i in range(len(ypred)):
  if(ypred[i]==0):
    Y_test.append('OBJ')
  else:
    Y_test.append('SUBJ')


In [ ]:
print(Y_test)

In [ ]:
f1 = open("Subj.y.test.txt", "w")
for y in Y_test:
  y1=str(y)
  f1.write(str(y1))
  f1.write("\n")

In [ ]:
f2=open("Subj.y.test.txt","r")
count=1
for x in f2:
  print(count,x)
  count+=1

1 OBJ

2 OBJ

3 OBJ

4 SUBJ

5 OBJ

6 SUBJ

7 SUBJ

8 SUBJ

9 OBJ

10 SUBJ

11 SUBJ

12 SUBJ

13 SUBJ

14 OBJ

15 OBJ

16 OBJ

17 OBJ

18 OBJ

19 OBJ

20 OBJ

21 SUBJ

22 SUBJ

23 SUBJ

24 OBJ

25 SUBJ

26 SUBJ

27 OBJ

28 OBJ

29 SUBJ

30 OBJ

31 SUBJ

32 SUBJ

33 OBJ

34 OBJ

35 OBJ

36 SUBJ

37 SUBJ

38 SUBJ

39 SUBJ

40 SUBJ

41 OBJ

42 SUBJ

43 OBJ

44 OBJ

45 SUBJ

46 SUBJ

47 OBJ

48 OBJ

49 OBJ

50 SUBJ

51 OBJ

52 SUBJ

53 SUBJ

54 SUBJ

55 OBJ

56 SUBJ

57 OBJ

58 SUBJ

59 SUBJ

60 OBJ

61 OBJ

62 OBJ

63 SUBJ

64 OBJ

65 OBJ

66 OBJ

67 OBJ

68 SUBJ

69 OBJ

70 OBJ

71 SUBJ

72 SUBJ

73 OBJ

74 OBJ

75 OBJ

76 OBJ

77 SUBJ

78 SUBJ

79 OBJ

80 OBJ

81 SUBJ

82 OBJ

83 OBJ

84 OBJ

85 OBJ

86 SUBJ

87 OBJ

88 OBJ

89 OBJ

90 OBJ

91 OBJ

92 OBJ

93 SUBJ

94 SUBJ

95 SUBJ

96 OBJ

97 OBJ

98 SUBJ

99 OBJ

100 SUBJ

101 SUBJ

102 SUBJ

103 SUBJ

104 SUBJ

105 OBJ

106 OBJ

107 OBJ

108 OBJ

109 OBJ

110 SUBJ

111 OBJ

112 OBJ

113 SUBJ

114 SUBJ

115 OBJ

116 SUBJ

117 SUBJ

11